# Comparison of Neural Models 2

Following an experiment to compare neural models, we discovered odd results showing that bag of words could outperform embeddings. This experiment attempts to tweak the embeddings to show the expected results under the assumption that the problem is not the amount of data. If the problem is the amount of data we will investigate this in another experiment.

Again, we will show Bag of Words results:

In [5]:
from scripts import training_helpers
from scripts.cross_validate import run_cross_validate
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Embedding
from tensorflow.keras.regularizers import l2

import gensim

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
data_frame = training_helpers.get_data_frame()
raw_features = data_frame['review']
labels = [x for x in data_frame['deceptive']]

num_words = 20000
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(raw_features)
bow_features = tokenizer.texts_to_matrix(raw_features, mode='tfidf')

In [3]:
def get_ff_bow_model():
  model = Sequential([
      Dense(16, activation=relu, input_shape=(num_words,), kernel_regularizer=l2(0.01)),
      Dropout(0.25),
      Dense(8, activation=relu, kernel_regularizer=l2(0.01)),
      Dense(1, activation=sigmoid)
  ])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

ff_bow_scores = run_cross_validate(get_ff_bow_model, bow_features, labels, cv=10, verbose=0)

Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 294us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 252us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 236us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 288us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 233us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 246us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 262us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 218us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 215us/step
Fitting with:  (1440, 20000) labels (1440,)
160/160 [==============================] - 0s 2

In [8]:
model = gensim.models.KeyedVectors.load_word2vec_format("../data/GoogleNews-vectors-negative300.bin", binary=True)

In [10]:
word_vectors = model

In [11]:
word_vectors.vector_size

300

In [13]:
model["blarg"]

array([-0.05419922,  0.01708984, -0.00527954,  0.33203125, -0.25      ,
       -0.01397705, -0.15039062, -0.265625  ,  0.01647949,  0.3828125 ,
       -0.03295898, -0.09716797, -0.16308594, -0.04443359,  0.00946045,
        0.18457031,  0.03637695,  0.16601562,  0.36328125, -0.25585938,
        0.375     ,  0.171875  ,  0.21386719, -0.19921875,  0.13085938,
       -0.07275391, -0.02819824,  0.11621094,  0.15332031,  0.09082031,
        0.06787109, -0.0300293 , -0.16894531, -0.20800781, -0.03710938,
       -0.22753906,  0.26367188,  0.012146  ,  0.18359375,  0.31054688,
       -0.10791016, -0.19140625,  0.21582031,  0.13183594, -0.03515625,
        0.18554688, -0.30859375,  0.04785156, -0.10986328,  0.14355469,
       -0.43554688, -0.0378418 ,  0.10839844,  0.140625  , -0.10595703,
        0.26171875, -0.17089844,  0.39453125,  0.12597656, -0.27734375,
       -0.28125   ,  0.14746094, -0.20996094,  0.02355957,  0.18457031,
        0.00445557, -0.27929688, -0.03637695, -0.29296875,  0.19

In [12]:
feature_vectors = [[model[x] for x in review.split()] for review in raw_features]

KeyError: "word 'Schicago' not in vocabulary"

In [ ]:
def get_ff_wv_model():
  model_ff_wv = Sequential([
      Embedding(corpus_vocab_size, embedding_length, weights=[embedding_matrix], trainable=False,
                input_length=max_sequence_length),
      Flatten(),
      Dense(16, activation=relu, kernel_regularizer=l2(0.01)),
      Dropout(0.25),
      Dense(8, activation=relu, kernel_regularizer=l2(0.01)),
      Dense(1, activation=sigmoid)
  ])
  model_ff_wv.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model_ff_wv